In [3]:
import numpy as np
import pandas as pd
import xarray as xr
import os

path = "../temp/NetCDF_n/"
path_r = "../data/NSRDB_4km.nc"
files = os.listdir(path)
if ".DS_Store" in files: files.remove(".DS_Store")

In [5]:
xr.open_dataset(path + files[0])


<xarray.Dataset> Size: 6GB
Dimensions:                                      (time: 219144, lon: 81, lat: 3)
Coordinates:
  * time                                         (time) datetime64[ns] 2MB 19...
  * lon                                          (lon) float64 648B -100.5 .....
  * lat                                          (lat) float64 24B 19.41 ... ...
Data variables: (12/14)
    Temperature                                  (time, lat, lon) float64 426MB ...
    Dew Point                                    (time, lat, lon) float64 426MB ...
    DNI                                          (time, lat, lon) float64 426MB ...
    DHI                                          (time, lat, lon) float64 426MB ...
    GHI                                          (time, lat, lon) float64 426MB ...
    Relative Humidity                            (time, lat, lon) float64 426MB ...
    ...                                           ...
    Pressure                                     (time, lat, lon) float64 426MB ...
    Surface Albedo                               (time, lat, lon) float64 426MB ...
    Wind Direction                               (time, lat, lon) float64 426MB ...
    Wind Speed                                   (time, lat, lon) float64 426MB ...
    Global Horizontal UV Irradiance (280-400nm)  (time, lat, lon) float64 426MB ...
    Global Horizontal UV Irradiance (295-385nm)  (time, lat, lon) float64 426MB ...
Attributes:
    CDI:          Climate Data Interface version 2.1.0 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Tue May 28 05:13:35 2024: cdo -s -O collgrid /Volumes/DATA/...
    CDO:          Climate Data Operators version 2.1.0 (https://mpimet.mpg.de...

In [2]:
ds_i = []
for f in files: ds_i.append( xr.open_dataset(path + f) )

vars = list(ds_i[0].keys())

ds = xr.Dataset( coords = { "time": ds_i[0]["time"], "lon": ds_i[0]["lon"],
    "lat": np.concatenate( [ x["lat"].values for x in ds_i ] ) } )

for v in vars:
    ds[v] = ( ("time", "lat", "lon"),
        np.concatenate( [ x[v].values for x in ds_i], axis = 1 ) )

ds.to_netcdf(path_r)

: 

In [ ]:
# Benchmarks
# 2
# concat: 00:09:17
# numpy: 00:09:10
# cdo: 00:02:03
# 3
# cdo: 00:03:04
# 2to4
# concat
# 3
# concat
# numpy

# Tamaño
# 1: 10.33 GB
# 2: 9.94 GB
# 4

In [3]:
ds = []
for f in files: ds.append( xr.open_dataset(path + f) )
xr.concat(ds, dim = "lat").to_netcdf(path_r)

: 

In [ ]:
ds = []
for f in files: ds.append( xr.open_dataset(path + f) )
xr.concat(ds, dim = "lat").to_netcdf(path_r)

In [1]:
# Convierte de CSV a NetCDF.

import os
import sys

import numpy as np
import pandas as pd

import xarray as xr

# Datos de entrada
path_d = "/Volumes/DATA/temp/temp/NSRDB_4km/CSV/"
path_r = "/Volumes/DATA/temp/temp/NSRDB_4km/NetCDF/"
files = os.listdir(path_d)
if ".DS_Store" in files: files.remove(".DS_Store")

In [2]:
f="20.33_-97.22.csv"
lat = f[:5]
lon = f[6:-4]
print(f" Procesando coordenadas {lat}°N {lon}°W...    ", end = "\r")

if not os.path.exists(f"{path_r}{lat}_{lon}.nc"):
    df = pd.read_csv( path_d + f, index_col = "time",
        parse_dates = True ).reset_index()

    # Convertimos a Dataset.
    df["lat" ] = float(lat)
    df["lon"] = float(lon)
    ds = df.set_index( ["time", "lat", "lon"] ).astype( float
        ).round( decimals = 1 ).to_xarray()
    ds["lat"] = ds["lat"].assign_attrs( standard_name = "latitude",
            long_name = "Latitude", units = "degrees" )
    ds["lon"] = ds["lon"].assign_attrs( standard_name = "longitude",
            long_name = "Longitude", units = "degrees" )
    ds.to_netcdf( f"{path_r}{lat}_{lon}.nc" )

In [ ]:
import os

import numpy as np
import pandas as pd

import xarray as xr

import xesmf as xe

path_d = "../temp/"
files = ["WRF_miroc_1985_2014.nc", "NSRDB_2km_.nc"]
path_r = "/Volumes/DATA/data/WRF_miroc_1985_2014_2km.nc"

ds = []
for f in files: ds.append( xr.open_dataset(path_d + f) )
ds[1] = ds[1].isel({"lat": slice(3, -4), "lon": slice(1, -3)})
ds[0]["XLAT"] = ds[0]["XLAT"].isel({"west_east": 0})
ds[0]["XLONG"] = ds[0]["XLONG"].isel({"south_north": 0})
ds[0] = ds[0].rename_vars( { "XLAT": "lat",
    "XLONG": "lon", "XTIME": "time" } )
ds[0] = ds[0].swap_dims( { "south_north": "lat", 
    "west_east": "lon", "XTIME": "time" } )

# Cambiamos la resolución.
regridder = xe.Regridder( ds[0], ds[1], "conservative" )
ds_rg = regridder( ds[0], keep_attrs = True )
ds_rg.to_netcdf(path_r)

In [1]:
import os
import sys

import numpy as np
import pandas as pd

import xarray as xr

import xesmf as xe


i = sys.argv[1]
path_d = "../temp/WRF_miroc_1985_2014_2km/"
files = sorted(os.listdir(path_d))
if ".DS_Store" in files: files.remove(".DS_Store")

ds_i = []

lat_i = [0, 0, 0, 42, 42, 42, 84, 84, 84]
lat_f = [42, 42, 42, 84, 84, 84, 126, 126, 126]
lon_i = [0, 53, 106, 0, 53, 106, 0, 53, 106]
lon_f = [53, 106, 158, 53, 106, 158, 53, 106, 158]

for f in files:
    ds_i.append( xr.open_dataset( path_d + f ).isel(
        {"lat": slice(lat_i[i], lat_f[i]),
         "lon": slice(lon_i[i], lon_f[i])}
        ).swap_dims({"XTIME": "time"}) )

ds = xr.concat(ds_i, dim = "time").to_netcdf(
    f"../temp/WRF_miroc_1985_2014_2km_{i}.nc" )

In [1]:
# Obtiene las variables DNI, UVHI, y P_mp.

import os

import numpy as np
import pandas as pd

import xarray as xr

# Funciones trigonométricas.
def sin(x): return np.sin(np.radians(x))
def cos(x): return np.cos(np.radians(x))
def asin(x): return np.arcsin(x) * 180/np.pi
def acos(x): return np.arccos(x) * 180/np.pi

dims = ["time", "lat", "lon"]

# Huso horario.
# La información está en UTC.
TZ = 0

# Cargamos el archivo.
n = 8
path_d = "../temp/WRF_miroc_1985_2014_2km/"
path_r = "../temp/"

# Iteramos para todos los archivos.
files = os.listdir(path_d)
files.sort()
if ".DS_Store" in files: files.remove(".DS_Store")
for f in files[0:1]:
    
    with xr.open_dataset( path_d + f) as ds:
        
        # Eccentric anomaly of the earth in its orbit around the sun.
        ds["Day_Angle"] = 6.283185 * ( ds[dims[0]].dt.dayofyear - 1 ) / 365
        # Declinación.
        ds["Declination"] = ( ( 0.006918 - 0.399912 * np.cos(ds["Day_Angle"])
            + 0.070257*np.sin(ds["Day_Angle"])
            - 0.006758*np.cos(2*ds["Day_Angle"])
            + 0.000907*np.sin(2*ds["Day_Angle"])
            - 0.002697*np.cos(3*ds["Day_Angle"])
            + 0.00148*np.sin(3*ds["Day_Angle"]) ) * 180/np.pi )
        # Ecuación del tiempo.
        ds["Time_Equation"] = ( ( 0.000075 + 0.001868*np.cos(ds["Day_Angle"])
            - 0.032077*np.sin(ds["Day_Angle"])
            - 0.014615*np.cos(2*ds["Day_Angle"])
            -0.040849*np.sin(2*ds["Day_Angle"])) * 229.18 )
        # Longitud del punto subsolar.
        ds["lon_subs"] = -15 * ( ds[dims[0]].dt.hour - TZ
            + ds["Time_Equation"]/60 )
        # Ángulo horario.
        ds["Hour_Angle"] = ( 15 * ( ds[dims[0]].dt.hour - 12
            - 0.5 + ds["Time_Equation"]/60 + ((ds[dims[2]]-TZ*15)*4)/60 ) )
        ds = ds.drop_vars( "Time_Equation" )
        # Posiciones del analema solar.
        #ds["Sx"] = cos(ds["Declination"])*cos(ds["lon_subs"]-ds["lon"])
        #ds["Sy"] = ( cos(ds["lat"])*sin(ds["Declination"])
        #    - sin(ds["lat"])*cos(ds["Declination"])
        #    *cos(ds["lon_subs"]-ds["lon"]) )
        ds["Sz"] = ( sin(ds[dims[1]])*sin(ds["Declination"])
            - cos(ds[dims[1]])*cos(ds["Declination"])
            *cos(ds["lon_subs"]-ds[dims[2]]) )
        ds = ds.drop_vars( "lon_subs" )
        # Ángulo del cénit solar.
        ds["Zenith_Angle"] = acos(ds["Sz"])
        ds = ds.drop_vars( "Sz" )
        # Ángulo acimutal solar.
        ds["Azimuth_Angle"] = acos( ( sin(ds["Declination"])
            - cos(ds["Zenith_Angle"])*sin(ds[dims[1]]) )
        / ( sin(ds["Zenith_Angle"])*cos(ds[dims[1]]) ) )
        ds["Azimuth_Angle"] = ds["Azimuth_Angle"].where(
            ds["Hour_Angle"] < 0, 360 - ds["Azimuth_Angle"] )
        ds = ds.drop_vars( ["Declination", "Hour_Angle"] )
        # Masa de aire.
        ds["Air_Mass"] = ( 1/(cos(ds["Zenith_Angle"])
            + 0.15/(93.885 - ds["Zenith_Angle"])**1.253 )
            * ds["Pressure"]/101325 )
        ds["Air_Mass"] = ds["Air_Mass"].where( ds["Zenith_Angle"] < 85.5, 0 )
		# Extraterrestrial radiation factor.
        ds["F_etr"] = ( cos(ds["Zenith_Angle"]) * 
            ( 1.00011 + 0.034221*np.cos(ds["Day_Angle"])
            + 0.00128*np.sin(ds["Day_Angle"])
            + 0.000719*np.cos(2*ds["Day_Angle"])
            + 0.000077*np.sin(2*ds["Day_Angle"]) ) )
        # Extraterrestrial radiation.
        I_cs   = 1367 # W m-2.
        I_uvcs = 78   # W m-2. 
        ds["I_tr"    ] = I_cs   * ds["F_etr"]
        ds["I_etr_uv"] = I_uvcs * ds["F_etr"]
        ds = ds.drop_vars( ["Day_Angle", "F_etr"] )

		# NREL DISC Model: DNI from GHI (Maxwell, 1987).
        # Effective global horizontal transmittance.
        ds["Kt"] = ds["GHI"] / ds["I_tr"]
        ds["Kt"] = ds["Kt"].where( ds["Kt"] < 1, 1 )
        ds["Kt"] = ds["Kt"].where( ds["Air_Mass"] > 0, 0 )
        # Coeficientes.
        ds["A"] = ( -5.743 + 21.77*ds["Kt"]
            - 27.49*ds["Kt"]**2 + 11.56*ds["Kt"]**3 )
        ds["A_1"] = ( 0.512 - 1.56*ds["Kt"]
            + 2.286*ds["Kt"]**2 - 2.222*ds["Kt"]**3 )
        ds["A"] = ds["A"].where( ds["Kt"] > 0.6, ds["A_1"] )
        ds = ds.drop_vars( "A_1" )
        ds["B"] = ( 41.4 - 118.5*ds["Kt"]
            + 66.05*ds["Kt"]**2 + 31.9*ds["Kt"]**3 )
        ds["B_1"] = 0.37 + 0.962*ds["Kt"]
        ds["B"] = ds["B"].where( ds["Kt"] > 0.6, ds["B_1"] )
        ds = ds.drop_vars( "B_1" )
        ds["C"] = ( -47.01 + 184.2*ds["Kt"]
            - 222*ds["Kt"]**2 + 73.81*ds["Kt"]**3 )
        ds["C_1"] = -0.28 + 0.932*ds["Kt"] - 2.048*ds["Kt"]**2
        ds["C"] = ds["C"].where( ds["Kt"] > 0.6, ds["C_1"] )
        ds = ds.drop_vars( "C_1" )
        # Delta Kn.
        ds["D_Kn"] = ds["A"] + ds["B"] * np.exp( ds["C"] * ds["Air_Mass"] )
        ds = ds.drop_vars( ["A", "B", "C"] )
        # Direct beam atmospheric transmittance under clear-sky conditions.
        ds["Knc"] = ( 0.866 - 0.122*ds["Air_Mass"] + 0.0121*ds["Air_Mass"]**2
            - 0.000653*ds["Air_Mass"]**3 + 0.000014*ds["Air_Mass"]**4 )
        # Radiación normal directa.
        ds["DNI"] = ds["I_tr"] * ( ds["Knc"] - ds["D_Kn"] )
        ds["DNI"] = ds["DNI"].where( ds["Kt"] > 0, 0 ).where( ds["DNI"] > 0, 0
            ).astype(np.float32).transpose(dims[0], dims[1], dims[2])
        ds = ds.drop_vars( ["Knc", "D_Kn", "I_tr"] )

        # Reordenamos el Dataset.
        ds["DNI" ] = ds["DNI" ].assign_attrs( units = "W m-2" )

        # Guardamos el archivo.
        ds.to_netcdf(path_r + f, mode = "w" )

KeyError: "No variable named 'Pressure'. Variables on the dataset include ['Temperature', 'GHI', 'Wind Speed', 'time', 'lon', 'lat', 'Day_Angle', 'Zenith_Angle', 'Azimuth_Angle']"